In [1]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import json


# pred_file_1 = './llama_pred/SPIDER-TEST_SQL_0-SHOT_CTX-200_ANS-2048_Llama_7b.json'
# pred_file_2 = './llama_pred/SPIDER-TEST_SQL_1-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-2048_Llama_7b.json'
# pred_file_3 = './llama_pred/SPIDER-TEST_SQL_3-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-2048_Llama_7b_try2.json'
pred_file_1 = './llama_pred/BIRD-TEST_SQL_0-SHOT_CTX-200_ANS-2048_evidence_Llama_7b.json'
pred_file_2 = './llama_pred/BIRD-TEST_SQL_1-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_Llama_7b.json'
pred_file_3 = './llama_pred/BIRD-TEST_SQL_3-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_Llama_7b.json'

with open(pred_file_1 , 'r') as f:
    refrence_gen_prompt_response_file_byte = f.read()
    refrence_gen_prompt_response = json.loads(refrence_gen_prompt_response_file_byte)

pred_files = [ pred_file_1 , pred_file_2 , pred_file_3 ]
generated_prompt_response_list = []

for file in pred_files:
    with open(file , 'r') as f:
        generated_prompt_response_file_byte = f.read()
        generated_prompt_response = json.loads(generated_prompt_response_file_byte)
        generated_prompt_response_list.append(generated_prompt_response)

for i in range ( len(generated_prompt_response_list[0]['questions']) ): #number of questions we have in a dataset
    tokenized_responses = []
    for j in range( len( generated_prompt_response_list ) ): #number of ensemble files we have for each question
        tokenized_responses.append( word_tokenize( generated_prompt_response_list[j]['questions'][i]['response'] ) )
    blue_scores = []
    for j in range( len( generated_prompt_response_list ) ):
        temp_tokenized_responses = tokenized_responses.copy()
        tokenized_response = temp_tokenized_responses.pop(j)
        blue_score = sentence_bleu( temp_tokenized_responses , tokenized_response )
        blue_scores.append(blue_score)
    max_bleu_score_value = max(blue_scores)
    max_index_bleu_score = blue_scores.index(max_bleu_score_value)
    refrence_gen_prompt_response['questions'][i]['response'] = generated_prompt_response_list[max_index_bleu_score]['questions'][i]['response']
    refrence_gen_prompt_response['questions'][i]['prompt'] = generated_prompt_response_list[max_index_bleu_score]['questions'][i]['prompt']
with open('./llama_pred/ENSEMBLE_seqLevelVote_BIRD-TEST_SQL_evidence0_1_3-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-2048_Llama_7b.json' , 'w' )as f:
    json.dump(refrence_gen_prompt_response , f)
    

/opt/anaconda3/envs/yadegari_llama/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/anaconda3/envs/yadegari_llama/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/anaconda3/envs/yadegari_llama/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consid

In [5]:
from utils.post_process import get_exec_output
# db_dir = './DAIL-SQL/dataset/spider/database'
db_dir = './DAIL-SQL/dataset/bird/database'
# import asyncio
def query_to_db(query , db_id):
    db_path = f"{db_dir}/{db_id}/{db_id}"
    flag, denotation = get_exec_output(
            db_path,
            query)
    return flag, denotation
    

generated_prompts_file = './DAIL-SQL/dataset/process/BIRD-TEST_SQL_0-SHOT_CTX-200_ANS-2048/questions.json'
with open(generated_prompts_file , 'r') as f:
    generated_prompts_file_byte = f.read()
    generated_prompts = json.loads(generated_prompts_file_byte)

generated_response_file = './llama_pred/ENSEMBLE_seqLevelVote_BIRD-TEST_SQL_evidence0_1_3-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-2048_Llama_7b.json'
with open(generated_response_file , 'r') as f:
    generated_response_file_byte = f.read()
    generated_response = json.loads(generated_response_file_byte)

from utils.post_process import result_eq
execution_accuracy = 0
counter = 0
for q_unit_gen , q_unit_truth in zip(generated_response['questions'] , generated_prompts['questions']):
    pred_response = q_unit_gen['response']
    start_of_answer = q_unit_truth['prompt'].splitlines()[-1]
    ground_truth = start_of_answer + ' ' + q_unit_truth['response']
    db_id = q_unit_truth['db_id']
    flag1, denotation1 = query_to_db(pred_response , db_id) #flag has ('result' , [data in columns])
    flag2, denotation2 = query_to_db(ground_truth , db_id)
    if flag1[0] != 'result':
        is_equal = False
        # print(  counter , '-' , flag1[0] , ' --> ' , 'pred_response: ' , pred_response )
    elif 'ORDER BY' in ground_truth or 'order by' in ground_truth:
        is_equal = result_eq(flag1[1] , flag2[1] , order_matters=True)
    else:
        is_equal = result_eq(flag1[1] , flag2[1] , order_matters=False)
    execution_accuracy += is_equal
    counter += 1
print( execution_accuracy/len(generated_response['questions']) )

TypeError: object of type 'OperationalError' has no len()